# Stock pricer viewer app

This demo shows a series of ipywidgets bundled in a Voila template, configured with Bootstrap Vue components.

To run it, you must installed the following dependencies (not contained with the template).
* `ipyhc`: Highcharts Jupyter widgets
* `ipybootstrapvue`: Bootstrap Vue components as Jupyter widgets

Just install via `pip install <dep>`.

Run the "app" with Voila:
`voila Stock_price_demo.ipynb --template sample-table-graph`


In [29]:
import ipybootstrapvue as bs
import ipyhc as hc

## Import some open stock price dataset from OpenML

In [33]:
import pandas as pd
import io
import requests
csv = requests.get('https://www.openml.org/data/get_csv/3660/dataset_2209_stock.arff').content.decode()
df = pd.read_csv(io.StringIO(csv))
df.head()

,company1,company2,company3,company4,company5,company6,company7,company8,company9,company10
0,17.219,50.500,18.750,43.000,60.875,26.375,67.750,19.000,48.750,34.875
1,17.891,51.375,19.625,44.000,62.000,26.125,68.125,19.125,48.750,35.625
2,18.438,50.875,19.875,43.875,61.875,27.250,68.500,18.250,49.000,36.375
3,18.672,51.500,20.000,44.000,62.625,27.875,69.375,18.375,49.625,36.250
4,17.438,49.000,20.000,41.375,59.750,25.875,63.250,16.500,47.500,35.500


In [34]:
import datetime
df['date'] = df.index.map(lambda x: datetime.date(1988, 1, 1) + datetime.timedelta(days=x))
df.head()

,company1,company2,company3,company4,company5,company6,company7,company8,company9,company10,date
0,17.219,50.500,18.750,43.000,60.875,26.375,67.750,19.000,48.750,34.875,1988-01-01
1,17.891,51.375,19.625,44.000,62.000,26.125,68.125,19.125,48.750,35.625,1988-01-02
2,18.438,50.875,19.875,43.875,61.875,27.250,68.500,18.250,49.000,36.375,1988-01-03
3,18.672,51.500,20.000,44.000,62.625,27.875,69.375,18.375,49.625,36.250,1988-01-04
4,17.438,49.000,20.000,41.375,59.750,25.875,63.250,16.500,47.500,35.500,1988-01-05


## Configure a plot

In [35]:
import numpy as np

In [36]:
def make_hc_plot(column):
    data = [{
        'name': column,
        'data': df[['date', column]].values.tolist()
    }]
    return hc.Chart(data=data, options={'title':{'text': f'Stock value for {column}'}})

In [38]:
# Uncomment to test plot
# make_hc_plot('company1')

In [39]:
graph = bs.Container(_metadata={'mount_id': 'graph'},
                     fluid=True,
                     children=[])
graph

Container(fluid=True)

## Configure a table display 

In [40]:
table = bs.Table(_metadata={'mount_id': 'table'},
                 items=[],
                 striped=True,
                 hover=True,
                 small=True)
table

Table(hover=True, items=[], small=True, striped=True)

## Build the form for choosing the stock to display

In [41]:
selector = bs.FormSelect(_metadata={'mount_id': 'selector'},
                         v_model=None,
                         options=[{'value': x, 'text': x} for x in df.columns if x.startswith('company')])

selector

FormSelect(options=[{'value': 'company1', 'text': 'company1'}, {'value': 'company2', 'text': 'company2'}, {'va…

In [43]:
def on_btn_click(widget, event, data):
    # v_model attributes of widgets contain the current component state
    company = selector.v_model
    badge.children = [company]
    graph.children = [make_hc_plot(company)]
    df_extract = df[['date', company]].head(10).copy()
    df_extract['date'] = df_extract['date'].apply(str)
    table.items = df_extract.to_dict(orient='records')

btn = bs.Btn(children=["Refresh"])
btn.on_event('click', on_btn_click)
btn

Btn(children=['Refresh'])

In [44]:
badge = bs.Badge(children=["unknown"])

comp_title = bs.Container(_metadata={'mount_id': 'form'}, 
                          children=[
                              bs.Row(children=[bs.Col(children=['Choose company'])], class_="m-4"),
                              bs.Row(children=[bs.Col(children=[selector])], class_="m-4"),
                              bs.Row(children=[bs.Col(children=[btn])], class_="m-4"),
                              bs.Row(children=[bs.Col(children=['Displaying result for ', badge])], class_="m-4")
                          ])
comp_title

Container(children=[Row(children=[Col(children=['Choose company'])], class_='m-4'), Row(children=[Col(children…